# Density ratios at the Weddell Sea mooring

Rowan Brown, August 2025

Buoyancy frequency, in its most basic form, is $$ N^2 = -\frac{g}{\rho} \frac{d \rho}{d z}. $$

Recall, we can define density by $$ R(S,\theta,p) = \rho_0 - \rho_0 \alpha \left( \theta - \theta_r \right) + \rho_0 \beta \left(S - S_r \right) + \textrm{pressure term},$$

where the expansion coefficients $\alpha$ and $\beta$ account for changes in density due to changes in temperature and salinity (I believe the pressure term counterpart is effectively constant), i.e., $$ \alpha = \frac{1}{\rho} \frac{\partial R}{\partial \theta}, \quad \beta = - \frac{1}{\rho} \frac{\partial R}{\partial S}. $$

We can then rearrange the buoyancy frequency in terms of $\alpha$ and $\beta$, i.e., 
$$\begin{align}
    \frac{N^2}{g} &= -\frac{1}{\rho} \left( \frac{d R}{d z} \right) \\
                  &= -\frac{1}{\rho} \left( \frac{\partial R}{\partial \theta}\frac{\partial \theta}{\partial z} + \frac{\partial R}{\partial S}\frac{\partial S}{\partial z} + \frac{\partial R}{\partial p}\frac{\partial p}{\partial z} \right) \\
                  &= -\frac{1}{\rho} \left( \frac{\partial R}{\partial \theta}\frac{\partial \theta}{\partial z} + \frac{\partial R}{\partial S}\frac{\partial S}{\partial z} - \rho g \frac{\partial R}{\partial p} \right) \\
                  &= -\frac{1}{\rho} \frac{\partial R}{\partial \theta}\frac{\partial \theta}{\partial z} -\frac{1}{\rho} \frac{\partial R}{\partial S}\frac{\partial S}{\partial z} + g \frac{\partial R}{\partial p}\\
                  &= \alpha \frac{\partial \theta}{\partial z} + \beta \frac{\partial S}{\partial z} + g \frac{\partial R}{\partial p}.\\
\end{align}$$

We can now consider the buoyancy contributions of the temperature and salinity gradients. See Ruddick and Gargett (2003) and van Haren and Millot (2006) for more.

##### Case 1: Diffusive convection 

Consider a stable water column where salinity and temperature both *increase* with depth, i.e., cold and fresh over warm and salty,
$$\begin{align}
    \frac{\partial S}{\partial z} > 0 &\Rightarrow \textrm{stabilising}, \\
    \frac{\partial \theta}{\partial z} > 0 &\Rightarrow \textrm{destabilising}. 
\end{align}$$ 

In this case, because heat diffuses faster than salt, there is a net downward flux of density (i.e., up-gradient) due to the buoyancy effects of upward heat diffusion outweighing those of upward salt diffusion. This can drive convection in well-mixed layers. Often this results in staircases, where vertical fluxes are enhanced by (a) diffusion driving interfacial fluxes and (b) convection carrying the fluxes from one step to the next. Ruddick and Gargett (2003) note that diffusive convection processes may change in the future since they are strong functions of the **density ratio**, $R_{\rho}=\alpha \theta_z / \beta S_z $. (Maybe consider how the Weddell Sea might be changing and how this could play a role?)

##### Case 2: Salt fingering

Now consider a stable water column where both salinity and temperature *decrease* with depth, i.e., warm and salty over cold and fresh, 
$$\begin{align}
    \frac{\partial S}{\partial z} < 0 &\Rightarrow \textrm{destabilising}, \\
    \frac{\partial \theta}{\partial z} < 0 &\Rightarrow \textrm{stabilising}. 
\end{align}$$ 

In this arrangement, imagine you displace a warm and salty water parcel downward; it is lighter than the water around it, but also warmer and saltier, hence heat diffuses outward and the parcel cools. It is now colder and while holding almost the same amount of salt, meaning it can sink downward and the process can continue. This also works in reverse for cold and fresh water displaced upwards, which can become warmer and hence lighter.

As in the case of diffusive convection, the result is a net downward flux of density due to the strong downward salt flux outweighing the weaker downward heat flux (i.e., a downard-moving parcel's salt continues to sink but its heat is mostly diffused sideways). As with the diffusive convection case, we can consider the **density ratio**, $R_{\rho}=\alpha \theta_z / \beta S_z $. 

##### Additional cases

As near as I can tell, the additional cases are either (1) both the salinity and temperature gradients increase with depth (fresh and warm over salty and cold), which leads to a boring and stable water column, or (2) both the salinity and temperature gradients decrease with depth (cold and salty over warm and fresh), which leads to active convection. 

## The Weddell Sea mooring

I will consider the density ratio,
$$\begin{align}
    R_{\rho} &= \alpha \theta_z / \beta S_z\\
             &= \frac{ \alpha \frac{\partial \theta}{\partial z} }{ \beta \frac{\partial S}{\partial z} }\\
             &= \frac{ \frac{1}{\rho} \frac{\partial R}{\partial \theta} \frac{\partial \theta}{\partial z} }{ - \frac{1}{\rho} \frac{\partial R}{\partial S} \frac{\partial S}{\partial z} },\\
\end{align}$$
which can be calculated numerically. This will tell me which term is more important in my case and *maybe* explain the stratification and downward mixing process that we see in the time series. And, if not, it becomes a sort of minimisation problem relating to non-linear effects, wind mixing, a front, or something else. (At some point, though, you'll also definitely want to revisit all of the mooring processing scripts and the correcting methods that you used.)

In [1]:
# Necessary imports
import matplotlib.pyplot as plt
import mooring_analyses as ma
import xarray as xr 
import numpy as np 
import gsw 

Note to self: You're basically plotting something about the heat vs salinity terms in N2. VERY DEPENDANT ON THINGS LIKE P_ref and data calibration, so play around with these and see what you come up with. Also consider if we're within error bars.

In [12]:
# Plotting!
def ds_plot(ds, fp, annot, dr_ylims=(-1, 1)):

    fig, [[ax1, ax3], [ax2, ax4]] = plt.subplots(ncols=2, nrows=2,
                                                figsize=(10,5))

    ds['density_ratio'].sel(depth_mid=-87.5).plot(ax=ax1, c='purple', lw=0.4)
    ds['density_ratio'].sel(depth_mid=-172.5).plot(ax=ax2, c='purple', lw=0.4)
    ax1b = ax1.twinx()
    ax2b = ax2.twinx()
    ds['N2'].sel(depth_mid=-87.5).plot(ax=ax1b, c='black', lw=0.4)
    ds['N2'].sel(depth_mid=-172.5).plot(ax=ax2b, c='black', lw=0.4)

    ds['numerator'].sel(depth_mid=-87.5).plot(ax=ax3, c='red', lw=0.4)
    ds['numerator'].sel(depth_mid=-172.5).plot(ax=ax4, c='red', lw=0.4)
    ax3b = ax3.twinx()
    ax4b = ax4.twinx()
    ds['denominator'].sel(depth_mid=-87.5).plot(ax=ax3b, c='blue', lw=0.4)
    ds['denominator'].sel(depth_mid=-172.5).plot(ax=ax4b, c='blue', lw=0.4)
    ax3b.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    ax4b.ticklabel_format(axis='y', style='sci', scilimits=(0,0))

    plt.suptitle(
        ("Mooring buoyancy ratios, "
        r"$\alpha \frac{\partial CT}{\partial z}/\beta"
        r"\frac{\partial SA}{\partial z}$"),
        fontsize=20)

    for ax in [ax1, ax1b, ax2, ax2b, ax3, ax3b, ax4, ax4b]:
        ax.set_title('')
        ax.set_ylabel('')
        ax.set_xlabel('')

    ax1.set_ylabel("Density ratio")
    ax2.set_ylabel("Density ratio")
    ax1b.set_ylabel("Buoyancy freq.")
    ax2b.set_ylabel("Buoyancy freq.")
    ax3.set_ylabel(r"$\alpha \frac{\partial CT}{\partial z}$")
    ax4.set_ylabel(r"$\alpha \frac{\partial CT}{\partial z}$")
    ax3b.set_ylabel(r"$\beta \frac{\partial SA}{\partial z}$")
    ax4b.set_ylabel(r"$\beta \frac{\partial SA}{\partial z}$")

    depths = {ax1: 87.5, ax2: 172.5, ax3: 87.5, ax4: 127.5}
    for ax in [ax1, ax2, ax3, ax4]:
        ax.text(
            0.5,
            1.01,
            "Depth = "+str(depths[ax])+" m",
            transform=ax.transAxes,
            fontsize=12,
            ha='center',
            va='bottom'
        )

    ax1.yaxis.label.set_color('purple')
    ax2.yaxis.label.set_color('purple')
    ax1b.yaxis.label.set_color('black')
    ax2b.yaxis.label.set_color('black')
    ax3.yaxis.label.set_color('red')
    ax4.yaxis.label.set_color('red')
    ax3b.yaxis.label.set_color('blue')
    ax4b.yaxis.label.set_color('blue')

    plt.subplots_adjust(
        hspace=0.45,
        wspace=0.5,
        top=0.815,
        bottom=0.15,
        left=0.085,
        right=0.92
    )

    plt.text(
        -0.15,
        -0.4,
        annot,
        ha="left",
        va="bottom",
        fontsize=12,
        transform=ax2.transAxes,
    )

    ax1.set_ylim(dr_ylims[0][0], dr_ylims[0][1])
    ax2.set_ylim(dr_ylims[1][0], dr_ylims[1][1])

    plt.savefig(fp, dpi=600)
    plt.clf()

In [15]:
woa = {True: '_woa_', False: '_non-woa_'}
time = ["full_time_series", "sept_time_series"]
dt = ["daily", "2h"]

# Open the data
for corr in ["_woa_", "_non-woa_"]:
    for ti in ["full_time_series", "sept_time_series"]:
        for dt in ["daily", str("2h")]:
            ds = ma.open_mooring_data()
            if dt == "daily": 
                ds = ds.convert_to_daily()
                annot = "Daily, "
            else:
                annot = "2-hourly, "
            if corr == "_woa_":
                ds.correct_mooring_salinities()
                annot = annot + "WOA-corrected salinities"
            else:
                annot = annot + "non-WOA-corrected salinities"
            if ti == "sept_time_series":
                ds = ds.sel(time=slice('2021-09-01', '2021-09-30'))
            ds.append_gsw_vars()  # Try varying this!
            ds = ds.sel(depth=[-50,-125,-220])

            fp = "buoy_ratios_"+str(dt)+str(corr)+str(ti)+".png"
            print(fp)
            dr_ylims = {
                "buoy_ratios_2h_non-woa_full_time_series.png":
                    ((-5 , 5), (-100, 100)),
                "buoy_ratios_2h_non-woa_sept_time_series.png":
                    ((-1 , 5), (-1 , 5)),
                "buoy_ratios_2h_woa_full_time_series.png":
                    ((-10, 10), (-1, 1)),
                "buoy_ratios_2h_woa_sept_time_series.png":
                    ((-10, 10), (-1, 1)),
                "buoy_ratios_daily_non-woa_full_time_series.png":
                    ((0, 0.7), (-70, 70)),
                "buoy_ratios_daily_non-woa_sept_time_series.png":
                    ((-1 , 2), (-1 , 2)),
                "buoy_ratios_daily_woa_full_time_series.png":
                    ((-7 , 2), (0 , 1.3)),
                "buoy_ratios_daily_woa_sept_time_series.png":
                    ((-5, 5), (0, 1)),
            }

            # Calculate N2
            print("Calculating N2")
            N2, pmid = [], []  # gsw.Nsquared calcs N2 at mid points
            for n,t in enumerate(ds['time']):
                tmp1, tmp2 = gsw.Nsquared(
                    ds['SA'].isel(time=n),
                    ds['CT'].isel(time=n),
                    ds['p_from_z'],
                    lat=-69.0005
                )
                N2.append(tmp1)
                pmid.append(tmp2)
            print("N2 calculated")

            # Necessary because we're looking at mid points
            depth_mid = [(-50-125)/2, (-125-220)/2]

            ds = ds.assign(
                N2=(['time', 'depth_mid'], [i for i in N2]),
                depth_mid=(depth_mid),
                pmid=(('depth_mid'),pmid[0])
            )

            # Calculate SA and CT at mid depths
            # Note pmid, where we have N2, is halfway between the sensors
            print("Beginning gradient calcs")
            ds = ds.assign(
                SA_mid=(
                    ['time', 'depth_mid'],
                    ds['SA'].interp(depth=depth_mid).values
                ),
                CT_mid=(
                    ['time', 'depth_mid'],
                    ds['CT'].interp(depth=depth_mid).values
                )
            )

            # Calculate alpha, beta
            ds['alpha'] = gsw.alpha(ds['SA_mid'], ds['CT_mid'], ds['pmid'])
            ds['beta'] = gsw.beta(ds['SA_mid'], ds['CT_mid'], ds['pmid'])

            # Calculate gradients
            # Multiply by -1 because the depth diff method yields negatives
            # i.e., water is cold over warm -> dCT/dz should be positive
            # and water is fresh over salt -> dSA/dz should be positive
            ds = ds.assign(
                dCTdz=(
                    ['time', 'depth_mid'],
                    ((-1)*ds['CT'].diff('depth')/ds['depth'].diff(
                        'depth')).values
                ), 
                dSAdz=(
                    ['time', 'depth_mid'], 
                    ((-1)*ds['SA'].diff('depth')/ds['depth'].diff(
                        'depth')).values
                )
            )

            # Calculate density ratio
            ds['numerator'] = ds['alpha']*ds['dCTdz']
            ds['denominator'] = ds['beta']*ds['dSAdz']
            ds['density_ratio'] = ds['numerator']/ds['denominator']
            print("Completed gradient calcs")

            print("Beginning to plot the data, "+fp)
            ds_plot(ds, fp, annot, dr_ylims[fp])
            ds.close()
            print("Plot saved")

Beginning to open the mooring data
Note that Markus is dotting i's and crossing t's re. checking how the salinities were corrected
Mooring data opened
Resampling to daily
Done resampling to daily
Beginning to correct the mooring data
Mooring data corrected
Adding GSW variables to the mooring time series
GSW variables added to the mooring time series
buoy_ratios_daily_woa_full_time_series.png
Calculating N2
N2 calculated
Beginning gradient calcs
Completed gradient calcs
Beginning to plot the data, buoy_ratios_daily_woa_full_time_series.png
Plot saved
Beginning to open the mooring data
Note that Markus is dotting i's and crossing t's re. checking how the salinities were corrected
Mooring data opened
Beginning to correct the mooring data
Mooring data corrected
Adding GSW variables to the mooring time series
GSW variables added to the mooring time series
buoy_ratios_2h_woa_full_time_series.png
Calculating N2
N2 calculated
Beginning gradient calcs
Completed gradient calcs
Beginning to plot 

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>